In [1]:
import pandas as pd
import pytest
from mmf_sa.models import ModelRegistry
from omegaconf import OmegaConf

In [2]:
base_config = OmegaConf.create({
    "date_col": "ds",
    "target": "y",
    "group_id": "unique_id",
    "freq": "D",
    "prediction_length": 14,
    "metric": "smape",
    "input_size_factor": 2,
    "loss": "smape",
    "accelerator": "cpu",
    "active_models": ["NeuralForecastInformer"],
    "informer_params": {
        "input_size": 14,   
        "n_epochs": 5,
        "batch_size": 32,
        "learning_rate": 1e-3,
        "accelerator": "cpu"
    }
})

In [3]:
model_registry = ModelRegistry(base_config)
model = model_registry.get_model("NeuralForecastInformer")

/Users/scott.mckean/Library/Application Support/hatch/env/virtual/mmf-sa/o6PZDoNu/full/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-16 21:13:11,248	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-16 21:13:11,306	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Seed set to 1


TypeError: unsupported operand type(s) for %: 'NoneType' and 'int'

In [ ]:
from pathlib import Path
from datasetsforecast.m4 import M4
def _transform_group(df):
    unique_id = df.unique_id.iloc[0]
    _start = pd.Timestamp("2020-01-01")
    _end = _start + pd.DateOffset(days=int(df.count()[0]) - 1)
    date_idx = pd.date_range(start=_start, end=_end, freq="D", name="ds")
    res_df = pd.DataFrame(data=[], index=date_idx).reset_index()
    res_df["unique_id"] = unique_id
    res_df["y"] = df.y.values
    return res_df

y_df, _, _ = M4.load(directory=str(Path.home()), group="Daily")
_ids = [f"D{i}" for i in range(1, 10)]
y_df = (
    y_df.groupby("unique_id")
    .filter(lambda x: x.unique_id.iloc[0] in _ids)
    .groupby("unique_id")
    .apply(_transform_group)
    .reset_index(drop=True)
)